In [2]:
# !pip install happytransformer
# !pip install xformers
# !pip install accelerate -U

In [ ]:
# Check current memory of GPU first: nvtop
# Then go to the gpu 

In [1]:
pwd

'/ghdevhome/home/cpan/notebooks'

In [11]:
ls '/ghdevhome/home/cpan/notebooks/'

all_billing_pdfs.txt
breast_cancer_files.txt
breast_cancer_matching_files.pkl
breast_cancer_processed_file_paths.txt
copy_medical_files.ipynb
data_vectorized_hierarchical.pkl
en_core_web_sm-3.6.0.tar.gz
Evaluate_OCR_quality.ipynb
experiment_script.py
Generate_synthetic_patients_data.ipynb
hierarchical_clustering_hyperparameter_tuning.ipynb
hierarchical_clustering.ipynb
hrd_folders_with_frequency_above_one.pkl
log_files/
maxent_ne_chunker/
maxent_ne_chunker.zip
models/
mychoice_folders_with_frequency_above_one.pkl
mychoice_results.csv
ocr_job.sh*
OCRopus_script.ipynb
OCRopus_script.py
OCRopus_text_extraction.ipynb
punkt/
punkt.zip
test_results_extraction.ipynb
text_cleaning_and_deidentification.ipynb
vectorizer_hiearchical.pkl
words/
words.zip


In [3]:
import pandas as pd
import numpy as np

In [4]:
data_dir = '/ghdevhome/home/cpan/data/'
predictions_df = pd.read_csv(data_dir + 'train_docs_sm.csv')

In [5]:
predictions_df.head()

,file_name,patient_id,page_content,label
0,A0569166_MedRec1/image-026.txt,A0569166,2022 11 29 11 11 53 29 47 REDACTED REDACTED My...,1
1,A0273290_MedRec2/image-033.txt,A0273290,01 05 2022 3 51 51 PM PAGE 37 OF 45 REDACTED R...,0
2,A83264/image-014.txt,A83264,O1 26 718 11 12 FROM CBSN Tenaya 7027493 708 T...,0
3,A0588576_MedRec1/image-082.txt,A0588576,To 18772418203 Paae 085 of 108 2022 12 01 18 3...,1
4,A0340486/image-005.txt,A0340486,To 16173942606 Page 04 of 12 2021 03 31 00 38 ...,0


In [6]:
test_df = pd.read_csv(data_dir + 'test_docs_sm.csv')

In [7]:
filtered_data = predictions_df[predictions_df['label'] == 1]
print(filtered_data.shape)
useful_pages = filtered_data['page_content']

(69, 4)


In [8]:
# extract test result pages from test data 
test_data = test_df[test_df['label'] ==1]
test_pages = test_data['page_content']

In [9]:
import os
import json
import torch  # Import PyTorch for GPU support
from happytransformer import HappyGeneration, GENSettings
import pandas as pd

# Check if a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")

Using GPU: NVIDIA A100-SXM4-80GB


In [5]:
happy_gen = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-125M")
happy_gen.model.to(device)

08/27/2023 13:20:19 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
Using pad_token, but it is not set yet.
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

In [23]:
new_prompt_template = "Based on this medical page content,for example: '{}'. \n Generate a similar synthetic patients' medical page content:"
# Prepare data for different cancer types
all_prompts = []
# try 10 examples
for page_content in useful_pages[:10]:
  full_prompt = new_prompt_template.format(page_content)
  all_prompts.append(full_prompt)

In [24]:
all_prompts[0]

"Based on this medical page content,for example: '2022 11 29 11 11 53 29 47 REDACTED REDACTED MyRisk Genetic Result Name REDACTED REDACTED DOBESep 13 1983 Accession 04370339 BLD Report Date Sep 09 2022 BREAST CANCER RISKSCORE REDACTED The breast cancer RiskScore provides 5 year and remaining lifetime breast cancer risks based on an analysis of genetic markers combined with patient clinical and family history data The Technical Specifications summary http myriad com technical specifications describes the RiskScore eligibility criteria analysis method performance and interpretive criteria of this test Data from 149 biomarkers are analyzed during next generation sequencing NGS The allele status of these markers has been weighted to generate a polygenic odds ratio of 1 0 for this patient which is combined with clinical and family history information to generate the final RiskScore This odds ratio is adjusted for overlap between the risk captured by the biomarkers and the clinical factors a

In [25]:
# prepara test prompts 
all_test_prompts = []
# try 10 examples
for page_content in test_pages:
  full_prompt = new_prompt_template.format(page_content)
  all_test_prompts.append(full_prompt)

In [25]:
# try an example
example_prompt = all_prompts[0]
example_res = happy_gen.generate_text(example_prompt)
print(example_res.text)

08/27/2023 13:33:46 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0
08/27/2023 13:33:46 - INFO - happytransformer.happy_transformer -   Initializing a pipeline
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 '2022 11 29 11 53 29 47 REDACTED REDACTED MyRisk Genetic Result Name REDACTED REDACTED DOBESep 13 1983 Accession 04370339 BLD Report Date Sep 09 2022 BREAST CANCER RISKSC


### Improve Setting

In [27]:
args1 = GENSettings(no_repeat_ngram_size=2,temperature=0.7, do_sample=True)

In [28]:
example_res1 = happy_gen.generate_text(example_prompt, args=args1)
print(example_res1.text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 http://www.cis.nhs.uk/mw/data_pages/genetics/risk/RISK_scores.html The Genetic Risk Score is a score for the ability of a person to predict a disease


In [29]:
args2 = GENSettings(
    no_repeat_ngram_size=2,  # Prevent repetition of n-grams
    temperature=0.7,  # Controls randomness (higher values make it more random)
    top_k=50,  # Controls the number of next words to consider (higher values make it more deterministic)
    top_p=0.95,  # Controls the diversity of next-word distribution (higher values make it more diverse)
    max_length=4000,  # Maximum length of the generated text
    early_stopping=True  # Stop generation when an end sequence is encountered)
)
example_res2 = happy_gen.generate_text(example_prompt, args=args2)
print(example_res2.text)

/home/cpan/miniconda3/envs/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/cpan/miniconda3/envs/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/cpan/miniconda3/envs/myenv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to

 http://www.genetics.org/content/show.php/genetic-genomics-2.html The Genetic Risk Score (GRSS) is a measure of the probability of a patient being a genetic risk factor for a disease. The GRSS is based upon the proportion of patients with a given risk score that are at a higher risk than the patient's risk. This measure is used to determine the likelihood of an individual being at an increased risk of developing a specific disease or condition. GRS is also used in a number of other studies to assess the impact of genetics on health care. For example, the GRAS score is an indicator of how much genetic variation is present in an individuals' genetic profile.

The Risk score for the following genes is calculated by dividing the number (or number) of genes in each gene by its number in all genes. A gene is considered to be a risk if its risk scores are greater than or equal to the sum of its scores for all of those genes, or its score equals the total number. If the score of all the genes 

### Fewshot Training

In [10]:
from happytransformer import HappyGeneration, GENTrainArgs

In [12]:
useful_pages[0]

'2022 11 29 11 11 53 29 47 REDACTED REDACTED MyRisk Genetic Result Name REDACTED REDACTED DOBESep 13 1983 Accession 04370339 BLD Report Date Sep 09 2022 BREAST CANCER RISKSCORE REDACTED The breast cancer RiskScore provides 5 year and remaining lifetime breast cancer risks based on an analysis of genetic markers combined with patient clinical and family history data The Technical Specifications summary http myriad com technical specifications describes the RiskScore eligibility criteria analysis method performance and interpretive criteria of this test Data from 149 biomarkers are analyzed during next generation sequencing NGS The allele status of these markers has been weighted to generate a polygenic odds ratio of 1 0 for this patient which is combined with clinical and family history information to generate the final RiskScore This odds ratio is adjusted for overlap between the risk captured by the biomarkers and the clinical factors and has not been validated for use with other risk

In [14]:
print(len(useful_pages))

69


In [18]:
new_prompt_template = "Based on this medical page content. For example, '{}' \n Generate a similar synthetic patients' medical page content:"
useful_pages_to_train = useful_pages[:10]  # Your list of useful pages

# Path to the training data file
train_data_path = '/ghdevhome/home/cpan/gpt_neo_train_train_test_results_pages.txt'

# Open the training data file for writing
with open(train_data_path, "w", encoding="utf-8") as train_file:
    for page_content in useful_pages:
        # Write the target text to the file
        train_file.write(f"{page_content}\n")

In [13]:
# # Prepare the train.txt data with all the test results from test results 
# # use 10 examples
# train_data = [f"{prompt}\n" for prompt in useful_pages][:10]  # Add "\n" at the end of each prompt

In [19]:
training_args = GENTrainArgs(
    num_train_epochs=2
)

In [21]:
torch.cuda.empty_cache()

In [22]:
model1 = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-125M")
model1.model.to(device)
model1.train(train_data_path, args =training_args)

checkpoint_dir = "/ghdevhome/home/cpan/models/gpt_neo_checkpoint2"
happy_gen.save(checkpoint_dir)

08/29/2023 10:15:22 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
Using pad_token, but it is not set yet.
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
08/29/2023 10:15:22 - INFO - happytransformer.happy_transformer -   Tokenizing training data...


Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

08/29/2023 10:15:23 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 6; 79.35 GiB total capacity; 1.05 GiB already allocated; 5.19 MiB free; 1.07 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [23]:
trained_model = HappyGeneration("GPT-NEO", "/ghdevhome/home/cpan/models/gpt_neo_checkpoint")

08/29/2023 10:15:47 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 50257. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


In [24]:
trained_model = model1

### Different Test Prompts Experimentation

In [71]:
# raw test prompt without any context 
test_prompt1 = 'Generate synthetic patient page content: '

In [73]:
example_res_trained1 = trained_model.generate_text(test_prompt1, args=args2)
print(example_res_trained1.text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




A:
   <script src="https://ajax.googleapis.com/aj�/cloud-native/v3/lib/jquery/1.11.3.min.js"></script>
<script type="text/javascript">
$(document).ready(function(){
var $ = $('#my-page');
if(!$){
alert('No page loaded');  //alert("No pages loaded");
}
});
</script></head>

  </body> 




In [77]:
all_test_prompts[3]

"Based on this medical page content,for example: 'PATIENT COPY conripentiat eee 53737937 myRisk Genetic Result Name REDACTED REDACTED DOB Aug 16 1969 Accession 03568288 BLD Report Date Fab 24 2020 Please contact REDACTED Medical Services at 1 800 469 7423 K 3850 to discuss any questions regarding this result These test results should only be used in conjunetion with the patient s cb i history and any previous analysis of appropriate family members The pahant s clinical history and test results should not be disclosed to a third party unle related to treatment or payment for treatment without the patient s express writer authorization It is strongly recommended hat these results be comrounmated to the patient in a setting that inciudes appropriate genatic consultation REDACTED lest was developed and its performance characteristics determined by REDACTED Genetic Laboratorigs It has not been cleared or approved by the REDACTED Food and Drug Administration FDA The FDA has determined that c

In [78]:
# test prompt with more context (but providing the test data) 
example_test_prompt = all_test_prompts[3]

In [82]:
print(len(all_test_prompts))

10


In [79]:
example_res_trained2 = trained_model.generate_text(example_test_prompt, args=args2)
print(example_res_trained2.text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




This page contains the following text:  

  1.  The patient is a patient of the FDA.
  2. The physician is the physician of REDACHANTI.


The patient's medical history is provided in this page. This page is intended to be a summary of a physician's clinical experience. It does not include any information about the medical condition of any patient. If the doctor is unable to provide a medical opinion, the information provided by a doctor may be considered confidential. REDATHAN MEDICAL SERVICES, INC. is an authorized physician in California.


In [80]:
# test prompt by mentioning competitors' test names 
test_prompt2 = "For the competitor's test as 'MyRisk'. Generate synthetic patient page content: "

In [81]:
example_res_trained3 = trained_model.generate_text(test_prompt2, args=args2)
print(example_res_trained3.text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




A:
   <div class="row">
<div>
</div><!-- /row -->
...
<?php
$row = mysql_fetch_array($query);
if (mysql_num_rows($row) > 0) {
?>




In [36]:
# prepara test prompts 
new_prompt_template2 = "Generate a synthetic medical page content similar to the provided example: {}"
all_test_prompts_new = []
# try 10 examples
for page_content in test_pages:
  full_prompt = new_prompt_template2.format(page_content)
  all_test_prompts_new.append(full_prompt)

In [37]:
all_test_prompts_new[0]

'Generate a synthetic medical page content similar to the provided example: CTCA Rightfax 1 22 2021 1 35 05 PM PAGE 9 040 Fax b5erver JACKSON REDACTED 6 15 2020 SEP20 534 REDACTED REDACTED 46263197 DOB AUG O1 1994 f Test Number 4 Definitions Somatic Afterations Not Detected ND or variant Characteristics may re dees not pr be present that are below the limit of detaction of this tivity The absence of delectable somatic alterations in circuiatir tumor in sampte 2 ONA interpretations Somatic altera types af yenum ions were NOT detected in th DNA isolated from this patient alterations chetented by GuardantséG This versiue of the Quardantsad tes is of gonomic alterations such as complex rearrangements or gene deletions nlood specirn not validated for toes the other types GUARBANT HE jad Guardtant360 Patient Report antine portat TST PRT 001 V2 Bortal quardanthealth com of to set up an aecount Gontaet Client Services 885 858 REDACTED REDACTED JONNA Page 4 of 5'

In [45]:
# example_res_trained4 = trained_model.generate_text(all_test_prompts_new[0], args=args2)
# print(example_res_trained4.text)

### Test Resulst Generation & Evaluation

In [38]:
args2 = GENSettings(
    no_repeat_ngram_size=2,  # Prevent repetition of n-grams
    temperature=0.7,  # Controls randomness (higher values make it more random)
    top_k=50,  # Controls the number of next words to consider (higher values make it more deterministic)
    top_p=0.95,  # Controls the diversity of next-word distribution (higher values make it more diverse)
    max_length=2000,  # Maximum length of the generated text
    early_stopping=True, # Stop generation when an end sequence is encountered
)

In [39]:
trained_model.model.to(device)

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

In [40]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')  # For GPU
torch.set_default_tensor_type('torch.FloatTensor')  #

In [42]:
all_test_prompts_new[0]

'Generate a synthetic medical page content similar to the provided example: CTCA Rightfax 1 22 2021 1 35 05 PM PAGE 9 040 Fax b5erver JACKSON REDACTED 6 15 2020 SEP20 534 REDACTED REDACTED 46263197 DOB AUG O1 1994 f Test Number 4 Definitions Somatic Afterations Not Detected ND or variant Characteristics may re dees not pr be present that are below the limit of detaction of this tivity The absence of delectable somatic alterations in circuiatir tumor in sampte 2 ONA interpretations Somatic altera types af yenum ions were NOT detected in th DNA isolated from this patient alterations chetented by GuardantséG This versiue of the Quardantsad tes is of gonomic alterations such as complex rearrangements or gene deletions nlood specirn not validated for toes the other types GUARBANT HE jad Guardtant360 Patient Report antine portat TST PRT 001 V2 Bortal quardanthealth com of to set up an aecount Gontaet Client Services 885 858 REDACTED REDACTED JONNA Page 4 of 5'

In [43]:
generated_texts = []
for test_prompt in all_test_prompts_new[:5]:
    gen_res = trained_model.generate_text(test_prompt, args=args2)
    generated_text = gen_res.text.to(device)
    generated_texts.append(generated_text)

    # Print device information for tensors
    print(f"Generated text device: {gen_res.text.device if hasattr(gen_res.text, 'device') else 'CPU'}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [89]:
reference_texts = test_pages.values

In [90]:
reference_texts[0]

'CTCA Rightfax 1 22 2021 1 35 05 PM PAGE 9 040 Fax b5erver JACKSON REDACTED 6 15 2020 SEP20 534 REDACTED REDACTED 46263197 DOB AUG O1 1994 f Test Number 4 Definitions Somatic Afterations Not Detected ND or variant Characteristics may re dees not pr be present that are below the limit of detaction of this tivity The absence of delectable somatic alterations in circuiatir tumor in sampte 2 ONA interpretations Somatic altera types af yenum ions were NOT detected in th DNA isolated from this patient alterations chetented by GuardantséG This versiue of the Quardantsad tes is of gonomic alterations such as complex rearrangements or gene deletions nlood specirn not validated for toes the other types GUARBANT HE jad Guardtant360 Patient Report antine portat TST PRT 001 V2 Bortal quardanthealth com of to set up an aecount Gontaet Client Services 885 858 REDACTED REDACTED JONNA Page 4 of 5'

In [95]:
# pip install rouge

In [94]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

In [ ]:
bleu_scores = []
rouge_scores = []

# Loop through generated and reference texts
for generated_text, reference_text in zip(generated_texts, reference_texts):
    # Compute BLEU score
    bleu_score = sentence_bleu([reference_text.split()], generated_text.split())
    bleu_scores.append(bleu_score)

    # Compute ROUGE scores
    rouge = Rouge()
    rouge_scores.append(rouge.get_scores(generated_text, reference_text)[0]['rouge-l']['f'])

# Calculate average BLEU and ROUGE scores
average_bleu = sum(bleu_scores) / len(bleu_scores)
average_rouge = sum(rouge_scores) / len(rouge_scores)

print(f"Average BLEU Score: {average_bleu:.4f}")
print(f"Average ROUGE Score: {average_rouge:.4f}")